In [42]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [43]:
# 데이터 불러오기
import pandas as pd
test = pd.read_csv("X_test.csv")
train = pd.read_csv("X_train.csv")
target = pd.read_csv("y_train.csv")

In [44]:
train.head(), test.head(), target.head()

(      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
 0   8440               B             Ship                    4   
 1  10849               D             Ship                    3   
 2   9478               B             Ship                    4   
 3   2968               B             Ship                    6   
 4   6964               B             Road                    5   
 
    Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
 0                1                  284                4                low   
 1                2                  201                4             medium   
 2                4                  181                3             medium   
 3                2                  236                4                low   
 4                1                  282                4                low   
 
   Gender  Discount_offered  Weight_in_gms  
 0      F                 4           4795  
 1      M               

In [45]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 756.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 

(None, None)

In [46]:
cols = train.select_dtypes(include="O").columns

from sklearn.preprocessing import LabelEncoder
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])

In [47]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   8799 non-null   int64
 1   Warehouse_block      8799 non-null   int64
 2   Mode_of_Shipment     8799 non-null   int64
 3   Customer_care_calls  8799 non-null   int64
 4   Customer_rating      8799 non-null   int64
 5   Cost_of_the_Product  8799 non-null   int64
 6   Prior_purchases      8799 non-null   int64
 7   Product_importance   8799 non-null   int64
 8   Gender               8799 non-null   int64
 9   Discount_offered     8799 non-null   int64
 10  Weight_in_gms        8799 non-null   int64
dtypes: int64(11)
memory usage: 756.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                  

(None, None)

In [48]:
train.shape, test.shape

((8799, 11), (2200, 11))

In [49]:
target_id = target.pop("ID")
train_id=  train.pop("ID")

In [50]:
test_id = test.pop("ID")

In [51]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(train, target)
pred = model.predict_proba(test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [52]:
pred[:,1]

array([0.39, 0.42, 0.41, ..., 0.5 , 0.35, 0.46])

In [53]:
submit = pd.DataFrame({"ID":test_id, "Reached.on.Time_Y.N":pred[:,1]}).to_csv("result.csv", index=False)

In [54]:
df = pd.read_csv("result.csv")
df

,ID,Reached.on.Time_Y.N
0,8285,0.39
1,10192,0.42
2,8675,0.41
3,5753,0.45
4,448,0.95
...,...,...
2195,7193,0.38
2196,4445,0.30
2197,7390,0.50
2198,9977,0.35
